# Exercise 1.3 - Solution
## Authors:
- Leonardo Kaplan 1212509
- Nino Fabrizio Tiriticco Lizardo 1113203

In [ ]:
# Pacotes utilizados
import pandas as pd # pandas: python data analysis
import numpy as np  # numpy: numerical analysis

# Premier League Season 2016_2017 - Solução dada em aula:

In [ ]:
# _EN read CSV file from the Web
# _PT lê arquivo CSV da Web

premierLeague2016_2017Raw = pd.read_csv('http://www.football-data.co.uk/mmz4281/1617/E0.csv')

# _EN check the dataframe dimensions
# _PT verifica as dimensões do dataframe

premierLeague2016_2017Raw.shape

In [ ]:
# _EN examine first few rows
# _PT examina as primeiras linhas

premierLeague2016_2017Raw.head()

In [ ]:
# _EN create a dataframe with selected columns
# _PT cria um dataframe com colunas selecionadas

pl = premierLeague2016_2017Raw.loc[ : , 'Date':'FTR']
pl.head()

In [ ]:
# _EN examine the column data types
# _PT examina os tipos de dados das colunas

pl.dtypes

In [ ]:
# _EN convert dates to datetime
# _PT converte as datas para datetime

pl.Date = pd.to_datetime(pl.Date)
pl.Date.head()

In [ ]:
pl.Date.tail()

In [ ]:
# _EN How many matches?
# _PT Quantas partidas?

pl.shape[0]

In [ ]:
# _EN How many teams?
# _PT Quantos times?

allTeams = list(set(pl['HomeTeam'].append(pl['AwayTeam'])))
print(len(allTeams))
sorted(allTeams)[0:5]

In [ ]:
# _EN How long (in days) is the championship?
# _PT Quantos dias dura o campeonato?

duracaoErro = max(pl['Date']) - min(pl['Date'])
print(duracaoErro)
print(duracaoErro.days)
print('certo')
pl.Date.iloc[-1]-pl.Date.iloc[0]

In [ ]:
# _EN create a sequential number for the match
# _PT cria um número sequencial para partidas

pl['Match'] = range(1, pl.shape[0]+1)

# _EN create column with winner's name, if any
# _PT cria coluna com o nome do vencedor, se houver

pl['Winner']= ''
pl['Winner'] = pl.apply(
    lambda row: row['HomeTeam'] if row['FTR'] == 'H' else row['Winner'], axis=1
)
pl['Winner'] = pl.apply(
    lambda row: row['AwayTeam'] if row['FTR'] == 'A' else row['Winner'], axis=1
)

# _EN create column with goal difference
# _PT cria coluna com saldo de gols

pl['GoalDiff'] = abs(pl['FTHG']-pl['FTAG'])

pl.head()

In [ ]:
# _EN sort rows in descending order by goal difference and ascending order by winner's name
# _PT ordena linhas por saldo de gols (maior para o menor) e nome do vencedor (em ordem alfabética)

pl.sort_values(['GoalDiff','Winner'], ascending=[False,True]).head()

In [ ]:
# _EN group by winner
# _PT agrupa por vencedor

plWinners = pl.groupby(['Winner'])
plWinners.groups

In [ ]:
# _EN How many victories per team? Which team had the most victories?
# _PT Quantas vitórias por time? Qual time teve mais vitórias?

victoryCount = plWinners.size().sort_values(ascending = False)
victoryCount

In [ ]:
# _EN create a dataframe with victory count
# _PT cria um dataframe com contagem de vitórias

plWin = pd.DataFrame(victoryCount, columns=['victories'])
plWin.head()

In [ ]:
# _EN calculate goal differences stats for each team
# _PT calcula estatísticas de saldo de gols para cada time

plWinners[['GoalDiff']].aggregate([np.min, np.median, np.mean, np.max])

### Melt

In [ ]:
# _EN melt dataframe to long format
# _PT remodela dataframe para formato "longo"

plLong = pd.melt(pl, id_vars=['Match','Date','HomeTeam', 'AwayTeam', 'Winner'], 
            value_vars=['FTHG', 'FTAG'], var_name='GoalPlace', value_name='GoalNum')
plLong.sort_values(['Match']).head()

In [ ]:
# _EN rename columns
# _PT altera nomes de colunas

plLong.columns = ['Match', 'Date', 'Home', 'Away', 'Winner', 'GoalPlace', 'GoalNum']
plLong['Location'] = plLong.Home
plLong.head()

In [ ]:
# _EN melt again
# _PT remodela novamente

plLong2 = pd.melt(plLong, 
                  id_vars = ['Match', 'Date', 'Location', 'Winner', 'GoalPlace', 'GoalNum'], 
            value_vars=['Home', 'Away'], var_name='Place', value_name='Team')
# _EN 
# _PT 

plLong2.sort_values(['Match','Place'], ascending=[True, False], inplace=True)

# _EN 
# _PT 

plLong2[['Match','Date', 'Location', 'Team','Place','GoalNum','GoalPlace','Winner']].head()

In [ ]:
# _EN 
# _PT 

plLong2.dtypes

In [ ]:
# _EN eliminate senseless rows ((Home and FTAG) or (Away and FTHG))
# _PT elimina linhas que não fazem sentido ((Home e FTAG) ou (Away e FTHG))

plLongOK = plLong2[((plLong2.Place=='Home') & (plLong2.GoalPlace=='FTHG'))|((plLong2.Place=='Away') & (plLong2.GoalPlace=='FTAG'))]
plLongOK = plLongOK[['Match', 'Date', 'Location', 'Team', 'Place', 'GoalNum']]
plLongOK.head()

In [ ]:
# _EN create a pivot dataframe with multi-index (w ERROR)
# _EN (original dataframe is not complete; needs filling)
# _PT cria dataframe pivot com multi-índice (com ERRO)
# _PT (dataframe original não está completo; precisa preencher)

plLongOK.pivot(index=['Match', 'Date', 'Location', 'Team'], columns='Place', values='GoalNum')

In [ ]:
pl.head()

In [ ]:
# _EN create a pivot dataframe with multi-index, filling with 0
# _PT cria dataframe pivot com multi-índice, preenchendo com 0

plPivot = pd.pivot_table(plLongOK, 
                         index=['Match', 'Date', 'Location', 'Team'],
                         columns=['Place'], values='GoalNum', fill_value=0)
plPivot.head()

_EN Does plPivot make sense? No, because the Away/Home columns are mutually exclusive.

_PT O dataframe plPivot faz sentido? Não, porque as colunas Away/Home são mutuamente exclusivas.

In [ ]:
# _EN create column with number of goals
# _PT cria coluna com número de goals

plPivot['Goals'] = plPivot.apply(lambda x: max(x['Away'], x['Home']), axis=1)
plPivot.head()

In [ ]:
# _EN eliminate redundant columns
# _PT elimina colunas redundantes

plPivot = plPivot[['Goals']]
plPivot.head()

In [ ]:
# _EN remove index
# _PT remove índice

plPivot = plPivot.reset_index()
plPivot.head()

In [ ]:
# _EN Why is the index column named Place?
# _PT Por que a coluna de índice está com nome Place?

plPivot.columns

In [ ]:
# _EN changing the name of the index coiumn
# _PT alterando o nome da coluna de índice
col = plPivot.columns
col.name = ''
plPivot.columns = col

plPivot.head()

## _EN Exercise 1.3: generate the dataframe above following a better procedure

## _PT Exercício 1.3: gere o dataframe acima seguindo um procedimento melhor
<br/>
<br/>
**Obs.: Melhor tirar todo o código feito pela professora na versão final, deixando apenas o que precisamos para nossa solução**

In [ ]:
# A tabela que queremos gerar
plPivot

In [ ]:
# A tabela com os dados brutos "tratados"
pl.head()

In [ ]:
# A tabela com os dados brutos
premierLeague2016_2017Raw.tail()

In [ ]:
# Limpando os dados com o que nos interessa
ourPLData = premierLeague2016_2017Raw[premierLeague2016_2017Raw.columns[1:6]]

# Renomeando algumas das colunas para algo mais legível
ourPLData.columns.values[3] = 'HomeTeamGoals'
ourPLData.columns.values[4] = 'AwayTeamGoals'

# Adicionando coluna da partida
# Obs.: Essa linha parece ser a origem de um warning chato pra cacete que não consegui retirar, mesmo seguindo a dica dele
ourPLData['Match'] = range(1, premierLeague2016_2017Raw.shape[0]+1)

# Ordenando as colunas para a ordem desejada
cols = ourPLData.columns.tolist()
cols = cols[-1:] + cols[:-1]
ourPLData = ourPLData[cols]

# Passando dados em Date para formato data
ourPLData.Date = pd.to_datetime(ourPLData.Date)

ourPLData

## *TODO*
A ideia dos seguintes passos para formar nossa tabela final é separar cada linha de **ourPLData** em duas novas linhas que possuam cada uma o nome do time e o respectivo número de gols feitos por ele na partida. Ou seja, devemos formar "duplas" de linhas que representem uma partida por dupla. Cada dupla então tem em comum o número da partida, a data e o local onde o jogo aconteceu, tendo como particularidade o nome do time e quantos gols fez nessa partida (olhar a tabela final feita pela professora pra melhor entendimento: **plPivot**).

In [ ]:
# Criando dataframe final
ourPLPivot = pd.DataFrame(columns=('Match', 'Date', 'Location', 'Team', 'Goals'))
for i in range(0, len(ourPLData)):
    
    # Pegando dados de cada time por índice de partida
    homeTeam = []
    awayTeam = []
    homeTeam += [ourPLData[ourPLData.columns.values[0]].values[i], ourPLData[ourPLData.columns.values[1]].values[i], ourPLData[ourPLData.columns.values[2]].values[i], ourPLData[ourPLData.columns.values[2]].values[i], ourPLData[ourPLData.columns.values[4]].values[i]]
    awayTeam += [ourPLData[ourPLData.columns.values[0]].values[i], ourPLData[ourPLData.columns.values[1]].values[i], ourPLData[ourPLData.columns.values[2]].values[i], ourPLData[ourPLData.columns.values[3]].values[i], ourPLData[ourPLData.columns.values[5]].values[i]]
    
    # Passando os dados ao dataframe final através de temporário
    df = pd.DataFrame([homeTeam, awayTeam], columns=('Match', 'Date', 'Location', 'Team', 'Goals'))
    ourPLPivot = ourPLPivot.append(df)

# Ordenando tabela por ordem de partida primeiro, alfabética em segundo e finalizando por ordenar índices da tabela
# Obs.: Para poder atribuir a mudança no dataframe com sort_values, tive que atribuir o dataframe gerado pela função ao original. A professora fez sem atribuir no material.
ourPLPivot = ourPLPivot.sort_values(['Match','Team'], ascending=[True,True])
ourPLPivot = ourPLPivot.reset_index(drop = True)
ourPLPivot